# What's in an Avocado Toast: A Supply Chain Analysis

![](../../imgs/avocado_wallpaper.jpeg)

You find yourself in London, crafting a delectable avocado toast, a dish that has risen dramatically in popularity on breakfast menus since the 2010s. This straightforward recipe requires just five ingredients: a ripe avocado, half a lemon, a generous pinch of salt flakes, two slices of sourdough bread, and a good drizzle of extra virgin olive oil. Most of these ingredients are now staples in grocery stores, and as you will find with this project, that is no small feat!

In this project, you'll conduct a supply chain analysis of three ingredients used in avocado toast using the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin.
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple category tags in the `categories_tags` column.
There is a column in each CSV file called `origins_tags`, which contains strings for the country of origin of each item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

[Open Food Facts database](https://world.openfoodfacts.org/)

In [13]:
import pandas as pd


In [14]:
## Their Code
avocado = pd.read_csv('../../data/avocado.csv', sep='\t')

# Subset large DataFrame to include only relevant columns
subset_columns = [ 'code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
avocado = avocado[subset_columns]

# Gather relevant categories data for avocados
with open("../../data/relevant_avocado_categories.txt", "r") as file:
    relevant_avocado_categories = file.read().splitlines()
    file.close()
    
### Filter avocado data using relevant category tags

# Turn a column of comma-separated tags into a column of lists
avocado['categories_list'] = avocado['categories_tags'].str.split(',')

# Drop rows with null values in a particular column
avocado = avocado.dropna(subset = 'categories_list')

# Filter a DataFrame based on a column of lists
avocado = avocado[avocado['categories_list'].apply(lambda x: any([i for i in x if i in relevant_avocado_categories]))]

### Where do most avocados come from?

# Filter DataFrame for UK data
avocados_uk = avocado[(avocado['countries']=='United Kingdom')]

# Find most common country for avocado origin
avocado_origin = (avocados_uk['origins_tags'].value_counts().index[0])
avocado_origin = avocado_origin.lstrip("en:")


### Create a general function to read and filter data for a particular ingredient, 
###    and return the top origin country for that food item

def read_and_filter_data(filename, relevant_categories):
  df = pd.read_csv('../../data/' + filename, sep='\t')
  
  # Subset large DataFrame to include only relevant columns
  subset_columns = [ 'code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
  df = df[subset_columns]

  # Split tags into lists
  df['categories_list'] = df['categories_tags'].str.split(',')

  # Drop rows with null categories data
  df = df.dropna(subset = 'categories_list')

  # Filter data for relevant categories
  df = df[df['categories_list'].apply(lambda x: any([i for i in x if i in relevant_categories]))]
    
  # Filter data for the UK
  df_uk = df[(df['countries']=='United Kingdom')]

  # Find top origin country string with the highest count
  top_origin_string = (df_uk['origins_tags'].value_counts().index[0])

  # Clean up top origin country string
  top_origin_country = top_origin_string.lstrip("en:")
  top_origin_country = top_origin_country.replace('-', ' ')

  print(f'**{filename[:-4]} origins**','\n', top_origin_country, '\n')

  print ("Top origin country: ", top_origin_country)
  print ("\n")

  # End of function - return top origin country for this ingredient
  return top_origin_country


# Analyze avocado origins again, this time by calling function
top_avocado_origin = read_and_filter_data('avocado.csv',relevant_avocado_categories)

### Repeat process above with new function for the other 2 ingredients

# Gather relevant categories data for olive oil
with open("../../data/relevant_olive_oil_categories.txt", "r") as file:
    relevant_olive_oil_categories = file.read().splitlines()
    file.close()

# Call user-defined function on olive_oil.csv
top_olive_oil_origin = read_and_filter_data('olive_oil.csv',relevant_olive_oil_categories)

# Gather relevant categories data for sourdough
with open("../../data/relevant_sourdough_categories.txt", "r") as file:
    relevant_sourdough_categories = file.read().splitlines()
    file.close()

# Call user-defined function on sourdough.csv
top_sourdough_origin = read_and_filter_data('sourdough.csv',relevant_sourdough_categories)

**avocado origins** 
 peru 

Top origin country:  peru




/tmp/ipykernel_625326/589788933.py:38: DtypeWarning: Columns (0,1,2,4,5,6,7,8,10,13,15,16,17,18,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,39,42,43,44,45,46,48,49,50,57,67,70,71,72,73,74,75,76,79,80,81,82,83,86,87,88,89,90,91,92,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,115,121,125,129,133,137,141,145,149,153,157,159,161,165,167,169,177,179,181,183,185,187,189,191,193,195,197,199,201,203,205,211,213,215,217,219,221,223,225,227,229,231,233,235,237,239,241,243,245,247,249,251,253,255,257,259,261,263,265,267,269,271,273,274,275,276,277,278,279,280,281,282,283,284,285,286,288,289,290,291,292,293,294,305,311,312,313,315,316,319,320,321,322,324) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/' + filename, sep='\t')


**olive_oil origins** 
 greece 

Top origin country:  greece


**sourdough origins** 
 united kingdom 

Top origin country:  united kingdom




In [30]:

## Define a function to clean/read our data
def read_and_clean_data(df_file_name, category_file_name, country, file_path="../../data", 
                        columns=['code', 'lc', 'product_name_en', 'quantity', 
                                 'serving_size', 'packaging_tags', 'brands', 
                                 'brands_tags', 'categories_tags', 'labels_tags', 
                                 'countries', 'countries_tags', 'origins',
                                 'origins_tags'],
                        sep="\t", low_memory=False
                        ):
#    print(f"======================= {df_file_name} =======================")
    ## Read the dataframe
    df = pd.read_csv(f"{file_path}/{df_file_name}", sep=sep, low_memory=low_memory)
    df = df[columns]
    
    ## Get the categories that we care about, from our files
    with open(f"{file_path}/{category_file_name}", "r") as file:
        categories = file.read().splitlines()
        file.close()

    ## Split up our category column
    df["category_list"] = df["categories_tags"].str.split(',')
    ## Drop NA's
    
#    print(df)
    df = df.dropna(subset="category_list")
    ## Filter the dataframe based upon the column of lists
    df = df[df["category_list"].apply(lambda x: any([i for i in x if i in categories]))]

    ## Filter down to the country we care about
    to_return = df[df["countries"] == country]
    to_return = to_return["origins_tags"].value_counts().index[0]
#    print(f"to_return: {to_return}")
    to_return = to_return.lstrip("en:")
#    print(f"to_return: {to_return}")
    to_return = to_return.replace('-', ' ')
#    print(f"to_return: {to_return}")

#    print("\n")
    
    return (to_return, df)
 
top_avocado_origin, avocado = read_and_clean_data("avocado.csv", "relevant_avocado_categories.txt", "United Kingdom")
top_olive_oil_origin, olive_oil = read_and_clean_data("olive_oil.csv", "relevant_olive_oil_categories.txt", "United Kingdom")
top_sourdough_origin, sourdough = read_and_clean_data("sourdough.csv", "relevant_sourdough_categories.txt", "United Kingdom")

print(f"Top Avocada country of origin = {top_avocado_origin}")
print(f"Top Olive Oil country of origin = {top_olive_oil_origin}")
print(f"Top Sourdough country of origin = {top_sourdough_origin}")


Top Avocada country of origin = peru
Top Olive Oil country of origin = greece
Top Sourdough country of origin = united kingdom
